In [1]:
import os
import pandas as pd


In [ ]:
filepath = "Resources/MUP_INP_RY24_P03_V10_DY22_PrvSvc.csv"
codefilepath = "Resources/DRG_to_MDC_Crosswalk.xlsx"
rucafilepath = "Resources/ruca_codes.csv"

fulldata = pd.read_csv(filepath, encoding='ISO-8859-1')
codedata = pd.read_excel(codefilepath)
rucadata = pd.read_csv(rucafilepath)

fulldata.head()

In [3]:
validdata = fulldata.drop(fulldata.columns[[0,1,2,3,4,8,10]], axis=1)


In [ ]:
validdata = validdata.rename(columns={
    'Rndrng_Prvdr_Zip5': 'zip',
    'Rndrng_Prvdr_State_Abrvtn': 'state',
    'Rndrng_Prvdr_RUCA': 'RUCA',
    'Tot_Dschrgs': 'discharges',
    'Avg_Submtd_Cvrd_Chrg': 'avg_covered_charge',
    'Avg_Tot_Pymt_Amt': 'avg_total_payment',
    'Avg_Mdcr_Pymt_Amt': 'avg_medicare_payment',
})
validdata.head()

In [ ]:

validdata['avg_oop'] = validdata['avg_total_payment']-validdata['avg_medicare_payment']

validdata.head()

In [ ]:
validdata = validdata.dropna(subset=['RUCA'])
validdata.info()

In [ ]:
#Prepare DRG to MDC code conversion file for merging by renaming DRG Code column 
codedata = codedata.rename(columns={"DRG_CODE":"DRG_Cd"})

#Merge validdata set with coding dataset to assign MDC codes to each column
alldata= pd.merge(codedata,validdata, how="left", on='DRG_Cd')
alldata.head()

In [ ]:
alldata.dtypes


In [ ]:
#Merge rucadata set with alldata dataset to assign MDC codes to each column
alldata= pd.merge(rucadata,alldata, how="left", on='RUCA')

alldata.head()

In [ ]:
alldata= alldata.drop(alldata.columns[[0,1,3]], axis=1)
alldata = alldata.rename(columns={"Description":"mdc_desc"})
alldata.head()

In [ ]:
alldata = alldata.rename(columns={"MDC_CODE":"mdc_code"})
newcolumnorder= ['state','zip','c_type','mdc_code','mdc_desc','discharges','avg_covered_charge','avg_total_payment','avg_medicare_payment','avg_oop']
alldata = alldata[newcolumnorder]
alldata['zip'] = alldata['zip'].astype(int)
alldata['discharges'] = alldata['discharges'].astype(int)
alldata.head()

In [ ]:
alldata.info()

In [16]:
alldata.to_csv('cleanedmedicaredata.csv', index=False)